<!-- metadata: title -->
# Kenya Unit Trusts: Money Market Fund(KES) Analysis

<!-- metadata: subtitle -->
> ### Can Kenyan Money Market Funds gurantee capital preservation? 

<!-- metadata: date, type=date -->
**Published Date:**
2024-03-03

<!-- metadata: date-modified, type=date-->
**Date Modified:**
2024-05-05

<!-- metadata: keywords, type=array -->
**Keywords:**
  - money
  - kenya
  - unit-trusts
  - money-market-funds
  - MMF

<!-- metadata: categories, type=array -->
**Categories:**
  - kenya-unit-trusts
  - money-market-funds

## Description

<!-- metadata: description -->
Money market is a form of unit trust, where fund managers collect money from the group of investors, and invest on their behave. This reduces the overhead of managing your portfio and significantly reduces your risk. Let's statistically and critically analyze Money market funds in Kenya in general using publicly available information and hopefully paint a clearer picture of the state of unit trusts in Kenya. 

- What are the risk factors that exist? 
- What is the performance of money market funds?

## Abstract

<!-- metadata: abstract -->
Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

## Introduction

In Kenya, a good number of the population have a bank account, even if not the traditional bank account. It is estimated that by year 2029, 99.9% of Kenyans will be banked, ^[Population share with banking account in Kenya 2014-2029
Published by J. Degenhard, Jan 30, 2024. <https://www.statista.com/forecasts/1149636/bank-account-penetration-forecast-in-kenya>]. With increase in financial access, financial literacy and regular individuals will want to venture in the teritory of finantial assets huunting for higher interest rates. One of the most attractive entry level high-yield financial asset is the unit trust, specifically the money market funds. Its easy to start, deposit, withdraw, and its interest is daily compunding, while offering higher interest than any bank.

## Analysis

### Imports

In [468]:
import pandas as pd

Let's start of by listing all the certified fund mangers in Kenya by CMA.^[Approved Fund Managers by CMA. <https://www.cma.or.ke/licensees-market-players/>]

In [469]:
import io
from bs4 import BeautifulSoup
from urllib.request import urlopen

html_content = urlopen("https://www.cma.or.ke/licensees-market-players/").read()
fund_managers_html_table = BeautifulSoup(html_content, "html.parser")\
    .find('span', string="FUND MANAGERS")\
        .parent\
            .parent\
                .parent\
                    .find('table')

fund_managers = pd.read_html(io.StringIO(str(fund_managers_html_table)))[0].dropna()
fund_managers

,NAME,ADDRESS,LICENCE NO.
0,Orient Asset Managers Limited,"P.O. Box 34530-00100, Nairobi",23.0
1,Amana Capital Limited,"P.O. Box 9480-00100, Nairobi",24.0
2,Apollo Asset Management Company Limited,"P.O. Box 30389-00100, Nairobi",25.0
3,Britam Asset Managers (Kenya) Limited,"P.O. Box30375-00100, Nairobi",27.0
4,Metropolitan Canon Asset Managers Limited,"P.O. Box 30216-00100, Nairobi",28.0
5,Nabo Capital Limited,"P.O. Box 10518-00100, Nairobi",29.0
6,CIC Asset Managers Limited,"P.O. Box 59485-00200, Nairobi",30.0
7,Co-op Trust Investment Services Limited,"P.O. Box 48231-00100, Nairobi",31.0
8,FCB Capital Limited,"P.O. Box 26219-00100, Nairobi",32.0
9,Fusion Investment Management Limited,"P.O. Box 47583-00100, Nairobi",33.0
